# 네이버 트렌드 데이터 분석## 행정동별 검색량 분석 (성수 기준 보정)이 노트북은 네이버 데이터랩에서 추출한 검색량 데이터를 분석합니다.- 데이터 기간: 2019.01.01 ~ 2025.09.30- 기준 키워드: '성수'- 분석 내용: 월별/쿼터별/년도별 합계 및 변화율

## 1. 라이브러리 설치 및 Import

In [ ]:
import pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport matplotlib.font_manager as fm# 한글 폰트 설정 (환경에 맞게 수정)# Windows: 'Malgun Gothic'# Mac: 'AppleGothic'# Linux: 'Noto Sans CJK JP' 또는 'NanumGothic'plt.rcParams['font.family'] = 'Malgun Gothic'  # 본인 환경에 맞게 수정plt.rcParams['axes.unicode_minus'] = Falseprint("라이브러리 로드 완료!")

## 2. 데이터 로드 및 구조 확인

In [ ]:
# 원본 데이터 로드df_raw = pd.read_excel('Naver_trend_rank.xlsx')print("=== 데이터 구조 ===")print(f"행 수: {len(df_raw)}, 열 수: {len(df_raw.columns)}")print("\n=== 컬럼명 ===")print(df_raw.columns.tolist())print("\n=== 처음 5행 ===")df_raw.head()

## 3. 성수 기준 데이터 보정네이버 데이터랩에서 한 번에 5개씩만 비교할 수 있어서 '성수'를 기준으로 여러 그룹으로 추출했습니다.각 그룹의 데이터를 '성수' 기준으로 통일하여 보정합니다.

In [ ]:
# 각 그룹별 성수 점수 확인seongsu_cols = ['성수', '성수.1', '성수.2', '성수.3', '성수.4', '성수.5']print("=== 각 그룹별 성수 점수 통계 ===")for col in seongsu_cols:    print(f"{col}: 평균={df_raw[col].mean():.2f}, 최소={df_raw[col].min():.2f}, 최대={df_raw[col].max():.2f}")

In [ ]:
# 성수 기준으로 보정# 기준이 될 성수 점수 (그룹2~6은 동일하므로 성수.1 사용)seongsu_base = df_raw['성수.1'].copy()# 결과 데이터프레임 생성df = pd.DataFrame()df['날짜'] = df_raw['날짜']df['성수'] = seongsu_base  # 성수는 그대로 (이미 기준)# 그룹1: 성수와 성수.1 비율로 보정ratio_g1 = seongsu_base / df_raw['성수']df['가산'] = df_raw['가산'] * ratio_g1df['서교동'] = df_raw['서교동'] * ratio_g1df['여의도'] = df_raw['여의도'] * ratio_g1df['압구정'] = df_raw['압구정'] * ratio_g1# 그룹2~6: 이미 성수.1 기준이므로 그대로df['상암'] = df_raw['상암']df['효창'] = df_raw['효창']df['삼성'] = df_raw['삼성']df['선정릉'] = df_raw['선정릉']df['선릉'] = df_raw['선릉']df['구로'] = df_raw['구로']df['서초'] = df_raw['서초']df['신당'] = df_raw['신당']df['마포'] = df_raw['마포']df['문정'] = df_raw['문정']df['논현'] = df_raw['논현']df['청담'] = df_raw['청담']df['금호'] = df_raw['금호']df['공덕'] = df_raw['공덕']df['송파'] = df_raw['송파']df['당산'] = df_raw['당산']df['서초2'] = df_raw['서초.1']df['방배'] = df_raw['방배']df['충정로'] = df_raw['충정로']print("=== 보정 완료 ===")print(f"전체 지역 수: {len(df.columns) - 1}")df.head()

In [ ]:
# 보정된 데이터 저장df.to_excel('Naver_trend_normalized.xlsx', index=False)print("보정된 데이터 저장 완료: Naver_trend_normalized.xlsx")

## 4. 날짜 처리 및 기간 컬럼 추가

In [ ]:
# 날짜 처리df['날짜'] = pd.to_datetime(df['날짜'])df['년도'] = df['날짜'].dt.yeardf['월'] = df['날짜'].dt.monthdf['년월'] = df['날짜'].dt.to_period('M')df['쿼터'] = df['날짜'].dt.to_period('Q')# 지역 컬럼만 추출region_cols = [col for col in df.columns if col not in ['날짜', '년도', '월', '년월', '쿼터']]print(f"분석 대상 지역 ({len(region_cols)}개):")print(region_cols)

## 5. 월별 / 쿼터별 / 년도별 합계 계산

In [ ]:
# 월별 합계monthly_sum = df.groupby('년월')[region_cols].sum()monthly_sum.index = monthly_sum.index.astype(str)print("=== 월별 합계 (처음 5행) ===")monthly_sum.head()

In [ ]:
# 쿼터별 합계quarterly_sum = df.groupby('쿼터')[region_cols].sum()quarterly_sum.index = quarterly_sum.index.astype(str)print("=== 쿼터별 합계 (처음 5행) ===")quarterly_sum.head()

In [ ]:
# 년도별 합계yearly_sum = df.groupby('년도')[region_cols].sum()print("=== 년도별 합계 ===")yearly_sum

## 6. 월별 / 쿼터별 / 년도별 변화율 계산

In [ ]:
# 월별 변화율 (전월 대비)monthly_pct = monthly_sum.pct_change() * 100print("=== 월별 변화율 (처음 5행) ===")monthly_pct.head()

In [ ]:
# 쿼터별 변화율 (전분기 대비)quarterly_pct = quarterly_sum.pct_change() * 100print("=== 쿼터별 변화율 (처음 5행) ===")quarterly_pct.head()

In [ ]:
# 년도별 변화율 (전년 대비)yearly_pct = yearly_sum.pct_change() * 100print("=== 년도별 변화율 ===")yearly_pct

## 7. 분석 결과 엑셀 저장

In [ ]:
# 엑셀 파일로 저장 (여러 시트)with pd.ExcelWriter('Naver_trend_analysis.xlsx', engine='openpyxl') as writer:    monthly_sum.to_excel(writer, sheet_name='월별_합계')    quarterly_sum.to_excel(writer, sheet_name='쿼터별_합계')    yearly_sum.to_excel(writer, sheet_name='년도별_합계')    monthly_pct.to_excel(writer, sheet_name='월별_변화율')    quarterly_pct.to_excel(writer, sheet_name='쿼터별_변화율')    yearly_pct.to_excel(writer, sheet_name='년도별_변화율')print("분석 결과 저장 완료: Naver_trend_analysis.xlsx")

## 8. 시각화 - 합계 시계열 차트

In [ ]:
# 주요 지역 선택 (2024년 기준 상위 8개)top_regions = yearly_sum.loc[2024].sort_values(ascending=False).head(8).index.tolist()print(f"주요 지역 (상위 8개): {top_regions}")# 색상 팔레트colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c', '#e67e22', '#34495e']

In [ ]:
# 합계 시계열 차트fig, axes = plt.subplots(3, 1, figsize=(16, 14))# 1. 월별 합계ax1 = axes[0]for i, region in enumerate(top_regions):    ax1.plot(range(len(monthly_sum)), monthly_sum[region], label=region, color=colors[i], linewidth=1.5, alpha=0.8)ax1.set_title('월별 검색량 합계 (성수 기준 지수)', fontsize=14, fontweight='bold')ax1.set_xlabel('월')ax1.set_ylabel('검색량 합계')ax1.legend(loc='upper left', ncol=4, fontsize=9)ax1.grid(True, alpha=0.3)xticks_monthly = list(range(0, len(monthly_sum), 12))ax1.set_xticks(xticks_monthly)ax1.set_xticklabels([str(monthly_sum.index[i]) for i in xticks_monthly], rotation=45)# 2. 쿼터별 합계ax2 = axes[1]for i, region in enumerate(top_regions):    ax2.plot(range(len(quarterly_sum)), quarterly_sum[region], label=region, color=colors[i], linewidth=2, marker='o', markersize=4)ax2.set_title('쿼터별 검색량 합계', fontsize=14, fontweight='bold')ax2.set_xlabel('분기')ax2.set_ylabel('검색량 합계')ax2.legend(loc='upper left', ncol=4, fontsize=9)ax2.grid(True, alpha=0.3)xticks_q = list(range(0, len(quarterly_sum), 4))ax2.set_xticks(xticks_q)ax2.set_xticklabels([str(quarterly_sum.index[i]) for i in xticks_q], rotation=45)# 3. 년도별 합계 (막대그래프)ax3 = axes[2]x = np.arange(len(yearly_sum))width = 0.1for i, region in enumerate(top_regions):    ax3.bar(x + i*width, yearly_sum[region], width, label=region, color=colors[i], alpha=0.8)ax3.set_title('년도별 검색량 합계', fontsize=14, fontweight='bold')ax3.set_xlabel('년도')ax3.set_ylabel('검색량 합계')ax3.set_xticks(x + width * 3.5)ax3.set_xticklabels(yearly_sum.index)ax3.legend(loc='upper left', ncol=4, fontsize=9)ax3.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.savefig('chart_sum_timeseries.png', dpi=150, bbox_inches='tight', facecolor='white')plt.show()

## 9. 시각화 - 변화율 시계열 차트

In [ ]:
# 변화율 시계열 차트fig, axes = plt.subplots(3, 1, figsize=(16, 14))# 1. 월별 변화율ax1 = axes[0]for i, region in enumerate(top_regions):    ax1.plot(range(len(monthly_pct)), monthly_pct[region], label=region, color=colors[i], linewidth=1, alpha=0.7)ax1.axhline(y=0, color='black', linestyle='--', linewidth=1)ax1.set_title('월별 변화율 (%)', fontsize=14, fontweight='bold')ax1.set_xlabel('월')ax1.set_ylabel('변화율 (%)')ax1.legend(loc='upper left', ncol=4, fontsize=9)ax1.grid(True, alpha=0.3)ax1.set_ylim(-100, 150)xticks_monthly = list(range(0, len(monthly_pct), 12))ax1.set_xticks(xticks_monthly)ax1.set_xticklabels([str(monthly_pct.index[i]) for i in xticks_monthly], rotation=45)# 2. 쿼터별 변화율ax2 = axes[1]for i, region in enumerate(top_regions):    ax2.plot(range(len(quarterly_pct)), quarterly_pct[region], label=region, color=colors[i], linewidth=2, marker='o', markersize=4)ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)ax2.set_title('쿼터별 변화율 (%)', fontsize=14, fontweight='bold')ax2.set_xlabel('분기')ax2.set_ylabel('변화율 (%)')ax2.legend(loc='upper left', ncol=4, fontsize=9)ax2.grid(True, alpha=0.3)xticks_q = list(range(0, len(quarterly_pct), 4))ax2.set_xticks(xticks_q)ax2.set_xticklabels([str(quarterly_pct.index[i]) for i in xticks_q], rotation=45)# 3. 년도별 변화율 (막대그래프)ax3 = axes[2]yearly_pct_plot = yearly_pct.dropna()x = np.arange(len(yearly_pct_plot))width = 0.1for i, region in enumerate(top_regions):    ax3.bar(x + i*width, yearly_pct_plot[region], width, label=region, color=colors[i], alpha=0.8)ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)ax3.set_title('년도별 변화율 (%)', fontsize=14, fontweight='bold')ax3.set_xlabel('년도')ax3.set_ylabel('변화율 (%)')ax3.set_xticks(x + width * 3.5)ax3.set_xticklabels(yearly_pct_plot.index)ax3.legend(loc='upper left', ncol=4, fontsize=9)ax3.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.savefig('chart_change_rate_timeseries.png', dpi=150, bbox_inches='tight', facecolor='white')plt.show()

## 10. 시각화 - 성수 집중 분석

In [ ]:
# 성수 집중 분석 차트fig, axes = plt.subplots(2, 2, figsize=(16, 12))# 1. 성수 월별 추이ax1 = axes[0, 0]ax1.fill_between(range(len(monthly_sum)), monthly_sum['성수'], alpha=0.3, color='#e74c3c')ax1.plot(range(len(monthly_sum)), monthly_sum['성수'], color='#e74c3c', linewidth=2)ax1.set_title('성수 월별 검색량 추이', fontsize=12, fontweight='bold')ax1.set_xlabel('월')ax1.set_ylabel('검색량')ax1.grid(True, alpha=0.3)xticks_m = list(range(0, len(monthly_sum), 12))ax1.set_xticks(xticks_m)ax1.set_xticklabels([str(monthly_sum.index[i]) for i in xticks_m], rotation=45)# 2. 성수 년도별 성장률ax2 = axes[0, 1]yearly_pct_seongsu = yearly_pct['성수'].dropna()colors_bar = ['#2ecc71' if x > 0 else '#e74c3c' for x in yearly_pct_seongsu]ax2.bar(yearly_pct_seongsu.index, yearly_pct_seongsu.values, color=colors_bar, alpha=0.8)ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)ax2.set_title('성수 연간 성장률 (%)', fontsize=12, fontweight='bold')ax2.set_xlabel('년도')ax2.set_ylabel('성장률 (%)')for i, (year, val) in enumerate(yearly_pct_seongsu.items()):    ax2.text(year, val + (2 if val > 0 else -5), f'{val:.1f}%', ha='center', fontsize=10, fontweight='bold')ax2.grid(True, alpha=0.3, axis='y')# 3. 년도별 히트맵ax3 = axes[1, 0]yearly_sum_t = yearly_sum[top_regions].Tim = ax3.imshow(yearly_sum_t.values, cmap='YlOrRd', aspect='auto')ax3.set_xticks(range(len(yearly_sum_t.columns)))ax3.set_xticklabels(yearly_sum_t.columns)ax3.set_yticks(range(len(yearly_sum_t.index)))ax3.set_yticklabels(yearly_sum_t.index)ax3.set_title('년도별 검색량 히트맵 (상위 8개 지역)', fontsize=12, fontweight='bold')plt.colorbar(im, ax=ax3, label='검색량')# 4. 2024년 지역별 순위ax4 = axes[1, 1]ranking_2024 = yearly_sum.loc[2024].sort_values(ascending=True)colors_rank = ['#e74c3c' if r == '성수' else '#3498db' for r in ranking_2024.index]ax4.barh(ranking_2024.index, ranking_2024.values, color=colors_rank, alpha=0.8)ax4.set_title('2024년 지역별 검색량 순위', fontsize=12, fontweight='bold')ax4.set_xlabel('검색량 합계')ax4.grid(True, alpha=0.3, axis='x')plt.tight_layout()plt.savefig('chart_seongsu_analysis.png', dpi=150, bbox_inches='tight', facecolor='white')plt.show()

## 11. 분석 결과 요약### 주요 인사이트1. **성수 트렌드**: 2019년부터 2023년까지 지속적인 성장 후, 2024년부터 하락 전환2. **2024년 검색량 순위**: 성수 > 서교동 > 서초 > 여의도 > 삼성3. **성장률 패턴**: 대부분의 지역이 비슷한 시기에 성장/하락 패턴을 보임### 생성된 파일- `Naver_trend_normalized.xlsx`: 성수 기준으로 보정된 데이터- `Naver_trend_analysis.xlsx`: 월별/쿼터별/년도별 합계 및 변화율- `chart_sum_timeseries.png`: 합계 시계열 차트- `chart_change_rate_timeseries.png`: 변화율 시계열 차트- `chart_seongsu_analysis.png`: 성수 집중 분석 차트